<a href="https://colab.research.google.com/github/gner007/ME462/blob/master/Egitim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [0]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Mon May  4 10:04:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': Operation not supported
 55e7e8af-ff39-475c-a558-980a4d7cfbcd.JPG
 659fb75d-722d-47b8-b280-31eb7e9171aa.JPG
 775c28c9-07cb-4c45-8e76-553465affadd.JPG
'7th Edition Solution Manuel.pdf'
'Adsız klasör'
 asf
 b987783d-da82-43dd-80a6-7e990c4438e8.JPG
'balans tezgahı.prt'
'Breaking Bad'
'çizimler staj'
'Çizim son hal'
'Colab Notebooks'
 CV_Mehmet_en.pdf
'CV_Ömer Şahin.pdf'
 D95CD4FE9F8A0587.pdf
'dp8 shaft.prt'
'Ev Çalışmaları'
'Funct Decom Morph Chart EXAMPLES by FORMER STUDENTS.rar'
 fundementals_of_mechanical_vibration_mcgraw_hill.pdf
 IMG_0192.MOV
 IMG_3167.JPG
 IMG_3168.JPG
 IMG_3169.JPG
 IMG_3170.JPG
 IMG_3171.JPG
 IMG_3172.JPG
 IMG_3173.JPG
 IMG_3174.JPG
 IMG_3175.JPG
 IMG_3176.JPG
 IMG_9899.JPG
 IMG_9900.JPG
'İngilizce _1557175301.pdf'
'[J._L._Meriam,_L._G._Kraige]_Engineering_Mechanics 2012(2

**1) Clone the Darknet**



In [0]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 13229, done.
remote: Total 13229 (delta 0), reused 0 (delta 0), pack-reused 13229
Receiving objects: 100% (13229/13229), 11.92 MiB | 24.32 MiB/s, done.
Resolving deltas: 100% (9050/9050), done.


**2) Compile Darknet using Nvidia GPU**


In [0]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused variabl

**3) Configure Darknet network for training YOLO V3**

In [0]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [0]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [0]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [0]:
!echo "Minder" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [0]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-05-04 10:06:36--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   263KB/s    in 7m 49s  

2020-05-04 10:14:26 (338 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [0]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/classes.txt    
  inflating: data/obj/IMG_1750.JPG   
  inflating: data/obj/IMG_1750.txt   
  inflating: data/obj/IMG_1751.JPG   
  inflating: data/obj/IMG_1751.txt   
  inflating: data/obj/IMG_1752.JPG   
  inflating: data/obj/IMG_1752.txt   
  inflating: data/obj/IMG_1753.JPG   
  inflating: data/obj/IMG_1753.txt   
  inflating: data/obj/IMG_1754.JPG   
  inflating: data/obj/IMG_1754.txt   
  inflating: data/obj/IMG_1755.JPG   
  inflating: data/obj/IMG_1755.txt   
  inflating: data/obj/IMG_1756.JPG   
  inflating: data/obj/IMG_1756.txt   
  inflating: data/obj/IMG_1757.JPG   
  inflating: data/obj/IMG_1757.txt   
  inflating: data/obj/IMG_1758.JPG   
  inflating: data/obj/IMG_1758.txt   
  inflating: data/obj/IMG_1759.JPG   
  inflating: data/obj/IMG_1759.txt   
  inflating: data/obj/IMG_1760.JPG   
  inflating: data/obj/IMG_1760.txt   
  inflating: data/obj/IMG_1761.JPG   
  inflating: data/obj/IMG_1761.txt   
  inflating: 

In [0]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.501072 0.433415 0.296262 0.414216

['15', '0.501072', '0.433415', '0.296262', '0.414216']
0 data/obj/IMG_1781.txt
0 0.501072 0.433415 0.296262 0.414216
15 0.443168 0.534722 0.234988 0.351307

['15', '0.443168', '0.534722', '0.234988', '0.351307']
1 data/obj/IMG_1827.txt
0 0.443168 0.534722 0.234988 0.351307
15 0.540135 0.418913 0.151961 0.249592

['15', '0.540135', '0.418913', '0.151961', '0.249592']
2 data/obj/IMG_1801.txt
0 0.540135 0.418913 0.151961 0.249592
15 0.531097 0.435662 0.279105 0.340278

['15', '0.531097', '0.435662', '0.279105', '0.340278']
3 data/obj/IMG_1825.txt
0 0.531097 0.435662 0.279105 0.340278
15 0.443168 0.562092 0.234988 0.348039

['15', '0.443168', '0.562092', '0.234988', '0.348039']
4 data/obj/IMG_1826.txt
0 0.443168 0.562092 0.234988 0.348039
15 0.580576 0.492443 0.373162 0.178513

['15', '0.580576', '0.492443', '0.373162', '0.178513']
5 data/obj/IMG_1785.txt
0 0.580576 0.492443 0.373162 0.178513
15 0.531097 0.524918 0.147365 0.225490

['15', '0.531097',

In [0]:
import glob
images_list = glob.glob("data/obj/*.JPG")
print(images_list)

['data/obj/IMG_1788.JPG', 'data/obj/IMG_1795.JPG', 'data/obj/IMG_1759.JPG', 'data/obj/IMG_1760.JPG', 'data/obj/IMG_1794.JPG', 'data/obj/IMG_1804.JPG', 'data/obj/IMG_1810.JPG', 'data/obj/IMG_1758.JPG', 'data/obj/IMG_1805.JPG', 'data/obj/IMG_1750.JPG', 'data/obj/IMG_1803.JPG', 'data/obj/IMG_1764.JPG', 'data/obj/IMG_1813.JPG', 'data/obj/IMG_1799.JPG', 'data/obj/IMG_1817.JPG', 'data/obj/IMG_1798.JPG', 'data/obj/IMG_1768.JPG', 'data/obj/IMG_1791.JPG', 'data/obj/IMG_1834.JPG', 'data/obj/IMG_1829.JPG', 'data/obj/IMG_1796.JPG', 'data/obj/IMG_1830.JPG', 'data/obj/IMG_1814.JPG', 'data/obj/IMG_1772.JPG', 'data/obj/IMG_1809.JPG', 'data/obj/IMG_1819.JPG', 'data/obj/IMG_1835.JPG', 'data/obj/IMG_1793.JPG', 'data/obj/IMG_1778.JPG', 'data/obj/IMG_1762.JPG', 'data/obj/IMG_1751.JPG', 'data/obj/IMG_1822.JPG', 'data/obj/IMG_1824.JPG', 'data/obj/IMG_1769.JPG', 'data/obj/IMG_1783.JPG', 'data/obj/IMG_1790.JPG', 'data/obj/IMG_1800.JPG', 'data/obj/IMG_1833.JPG', 'data/obj/IMG_1757.JPG', 'data/obj/IMG_1767.JPG',

In [0]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [0]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.867287, GIOU: 0.862592), Class: 0.999970, Obj: 0.983640, No Obj: 0.000194, .5R: 1.000000, .75R: 1.000000, count: 1, class_loss = 0.000067, iou_loss = 0.006629, total_loss = 0.006696 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000000, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 

 3903: 0.042872, 0.055887 avg loss, 0.001000 rate, 3.823646 seconds, 249792 images, 0.419050 hours left
Loaded: 4.136245 seconds - performance bottleneck on CPU or Disk HDD/SSD
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.848010, GIOU: 0.844943), Class: 0.998248, Obj: 0.964689, No Obj: 0.001937, .5R: 1.000000, .75R: 1.000000, count: 4, class_loss = 0.002927, iou_loss = 0.065338, total_loss = 0.068266 
v3 (mse 